In [1]:
import librosa as lr 
import numpy as np
import os

In [5]:
#--------整理noise data------------------
noise_list = []
for file in os.listdir('10_noise'):
    if '.wav' in file:
        noise_list.append(os.path.join('10_noise',file))
#----------------------------------------

In [6]:
noise_list[:5]

['10_noise\\ambulance_passing.wav',
 '10_noise\\breaking_concrete.wav',
 '10_noise\\breaking_houses2.wav',
 '10_noise\\breaking_houses4.wav',
 '10_noise\\car_road.wav']

In [7]:
#---------整理clean data------------------
filename_list = []
for filename in os.listdir('clean'):
    if '.' not in filename:
        filename_list.append(filename)
file_list = []
for filename in filename_list:    
    for file in os.listdir(os.path.join('clean',filename)):
        if '.wav' in file:
            file_list.append(os.path.join('clean',filename,file))
file_list.sort()
#------------------------------------------

In [8]:
filename_list[:5]

['amanda', 'angelababy', 'bandie', 'blue', 'chris']

In [11]:
#-----------隨機混音開始---------------------
if not os.path.isdir('mixed_all_snr'):
    os.mkdir('mixed_all_snr')
for file in file_list:
    for noise_index in range(len(noise_list)):
        snr = np.random.randint(-15,16)  #隨機取 snr的值

        # noise_index = np.random.randint(0,len(noise_list))
        cleanwav = file
        noisewav = noise_list[noise_index]

        noise,sr = lr.load(noisewav,sr=16000) # sr設置為None可取原檔案中的sample rate
        clean,Sr = lr.load(cleanwav,sr=16000)

        cleanenergy = np.sum(np.power(clean,2)) #np.power(a,2) = np.square(a)
        # 隨機索引與clean長度相同的noise信號
        ind = np.random.randint(1, len(noise) - len(clean) + 1)
        noiselen=noise[ind:len(clean) + ind]
        # 噪声语音能量  噪音的功率
        noiseenergy = np.sum(np.power(noiselen,2))
        # 噪声等级系数
        noiseratio = np.sqrt((cleanenergy / noiseenergy) / (np.power(10, snr * 0.1)))
        # 隨機索引與clean長度相同的noise信號
        noisyAudio = clean + noise[ind:len(clean)+ind] * noiseratio

        # write wav
        noisywavname='./mixed_all_snr/'+cleanwav.split('\\')[-1].rstrip('.wav')+'_'+noisewav.split('\\')[-1].rstrip('.wav')+"_snr"+str(snr)+".wav"

        # sf.write(noisywavname, noisyAudio, 16000)
        lr.output.write_wav(noisywavname,noisyAudio,16000)